In [17]:
import sys,string
from rdkit.Chem import AllChem as Chem
from optparse import OptionParser

#convert smiles to sdf
def getRMS(mol, c1,c2):
    (rms,trans) = Chem.GetAlignmentTransform(mol,mol,c1,c2)
    return rms


In [22]:
input = 'mol.smiles'
output = 'smifile = open(input)'
smifile = open(input)
for line in smifile:
    toks = line.split(',')
    smi = toks[0]
    name = "".join(toks[1:])    
    mol = Chem.MolFromSmiles(smi)

In [23]:
name

'c'

In [21]:
string

<module 'string' from '/home/shenwanxiang/anaconda3/envs/tf2/lib/python3.6/string.py'>

In [5]:


# if options.verbose:
#     print("Generating a maximum of",options.maxconfs,"per a mol")
    

sdwriter = Chem.SDWriter(output)
if sdwriter is None:
    print("Could not open ".output)
    sys.exit(-1)
    
for line in smifile:
    toks = line.split()
    smi = toks[0]
    name = string.join(toks[1:])    
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        if options.verbose:
            print(smi)
        try:
            Chem.SanitizeMol(mol)
            mol.SetProp("_Name",name);
            cids = Chem.EmbedMultipleConfs(mol, int(options.sample*options.maxconfs),randomSeed=options.seed)
            if options.verbose:
                print(len(cids),"conformers found" )
            cenergy = []            
            for conf in cids:
                #not passing confID only minimizes the first conformer
                converged = not Chem.UFFOptimizeMolecule(mol,confId=conf)
                if options.verbose:
                    print("Convergence of conformer",conf,converged)
                cenergy.append(Chem.UFFGetMoleculeForceField(mol,confId=conf).CalcEnergy())
            
            sortedcids = sorted(cids,key = lambda cid: cenergy[cid])
            if(options.rms == 0):
                cnt = 0;
                for conf in sortedcids:
                    if(cnt >= options.maxconfs):
                        break
                    if(options.energy < 0) or cenergy[conf]-cenergy[0] <= options.energy:
                        sdwriter.write(mol,conf)
                        cnt+=1 
            else:
                written = {}
                for conf in sortedcids:
                    if len(written) >= options.maxconfs:
                        break
                    #check rmsd
                    passed = True
                    for seenconf in written.iterkeys():
                        rms = getRMS(mol,seenconf,conf) 
                        if(rms < options.rms) or (options.energy > 0 and cenergy[conf]-cenergy[0] > options.energy):
                            passed = False
                            break
                    if(passed):
                        written[conf] = True
                        sdwriter.write(mol,conf)
        except (KeyboardInterrupt, SystemExit):
            raise                
        except:
            print("Exception occurred: ",sys.exc_info()[0])
    else:
        print("ERROR:",smi)

NameError: name 'smifile' is not defined

AttributeError: module 'string' has no attribute 'join'

In [10]:
toks

['CC[C@H](C)NS(=O)(=O)C1=CC=C2N=C(NC3=CC=C(C)C=C3)N([C@@H](C)C(C)(C)C)C2=C1']

In [26]:
from openforcefield.topology import Molecule
from openmmforcefields.generators import SystemGenerator

ModuleNotFoundError: No module named 'openforcefield'